## Burrito Builder Application Test Script
The tests below will generate known solutions for comparing with the results presented by the Pareto Front and verify that the source countries being displayed on the map are correct.<br>
For each test case, simply call the build_burrito_test function with the following parameters:
> - Data: The imported .json file<br>
> - home_country: The importing country or the country where the burrito is being built. defaults to Australia.<br>
> - ingredients: A list containing up to three ingredients. Defaults to Tortilla.<br>
> - budget: The amount of money that cannot be exceeded. Defaults to $10

**Here is the list of home_country that can be used in the function call.**<br>
<br>
['Australia', 'Brazil', 'Bolivia', 'Canada', 'Cabo Verde', 'Colombia',  'Czechia', 'Guyana', 'Luxembourg', 'Madagascar', 'Mauritius', 'Mexico',
       'Romania', 'Mozambique', 'South Africa', 'Serbia', 'Slovakia    'Uruguay', 'Turkey', 'Zambia', 'Argentina', 'Chile', 'Iceland',
       'Paraguay', 'Philippines', 'Tanzania', 'U

**Here is the list of ingredients that can be used in the function call.**<br>
<br>
['Beef', 'Pork', 'Chicken', 'Sour Cream', 'Cheese', 'Onions', 'Beans',  'Avocado', 'Rice', 'Tortilla', 'Salsa', 'Lettuce', 'Tomato']SA']

In [1]:
# from pandas import json_normalize
import pandas as pd
import numpy as np
burr_data = pd.read_json('final_burrito_data.json')
# ingred_data = pd.read_json('ingredient_data.json')
burr_data.index

Index(['Beef', 'Pork', 'Chicken', 'Sour Cream', 'Cheese', 'Onions', 'Beans',
       'Avocado', 'Rice', 'Tortilla', 'Salsa', 'Lettuce', 'Tomato'],
      dtype='object')

### Equations

>>> $Cost(i,\ s,\ h) = FoodCost(i,\ s) + TransCost(s,\ h)$<br>
<br>
>>> $Env_{Impact}(i, s, h) = \text{FoodImpact(i)} + \text(TransImpact(s,\ h))$

## Function

### Test cases: Function

In [2]:
def build_burrito_test(data, home_country='Australia', ingredients=['Tortilla'], budget=10):
    """
    This function will allow for testing up to three ingredients at a time. 
    Going beyond three ingredients takes a really long time to run.
    """
    assert type(home_country) is str
    assert type(data) is pd.DataFrame
    assert type(ingredients) is list
    pareto_front = []

    # Case 1: Single ingredient
    if len(ingredients) == 1:
        single_df = pd.DataFrame(data.loc[ingredients[0]][home_country])
        single_df.drop('Latitude', inplace=True)
        single_df.drop('Longitude', inplace=True)
        # Melt DataFrame
        cost = single_df.loc['cost']
        impact = single_df.loc['impact']
        final_df = pd.DataFrame({'cost': cost, 'impact': impact})
        budget_df = final_df[final_df['cost'] <= budget]
        budget_sorted = budget_df.sort_values(by=['cost'])
        # Get non-dominating solutions
        for i, row_i in budget_sorted.iterrows():
            is_dominated = False
            for j, row_j in budget_sorted.iterrows():
                if (row_j['cost'] <= row_i['cost'] and row_j['impact'] <= row_i['impact'] and 
                   (row_j['cost'] < row_i['cost'] or row_j['impact'] < row_i['impact'])):
                    is_dominated = True
                    break
            if not is_dominated:
                pareto_front.append(row_i)
        return pd.DataFrame(pareto_front)

    # Case 2: Two ingredients
    elif len(ingredients) == 2:
        combinations = []
        for ting_source, ting_values in data[home_country][ingredients[0]].items():                   
            for ning_source, ning_values in data[home_country][ingredients[1]].items():
                tot_cst = ting_values['cost'] + ning_values['cost']       
                tot_impt = ting_values['impact'] + ning_values['impact']
                if tot_cst <= budget:
                    combinations.append({ingredients[0] +'_'+'source': ting_source,
                                        ingredients[1] +'_'+ 'source': ning_source,
                                        'total cost': tot_cst,
                                        'total impact': tot_impt})
        combinations_df = pd.DataFrame(combinations)
        combinations_sorted = combinations_df.sort_values(by=['total cost'])
        for i, row_i in combinations_sorted.iterrows():
            comb_dominated = False
            for j, row_j in combinations_sorted.iterrows():
                if (row_j['total cost'] <= row_i['total cost'] and row_j['total impact'] <= row_i['total impact'] and
                    (row_j['total cost'] < row_i['total cost'] or row_j['total impact'] < row_i['total impact'])):
                    comb_dominated = True
                    break
            if not comb_dominated:
                pareto_front.append(row_i)
        out_df = pd.DataFrame(pareto_front)
        out_df.reset_index(drop=True, inplace=True)
        return out_df
        
    # Case 3: Three ingredients
    elif len(ingredients) == 3:
        combinations = []
        for ting_source, ting_values in data[home_country][ingredients[0]].items():                   
            for ning_source, ning_values in data[home_country][ingredients[1]].items():
                for ling_source, ling_values in data[home_country][ingredients[2]].items():
                    tot_cst = ting_values['cost'] + ning_values['cost'] + ling_values['cost']      
                    tot_impt = ting_values['impact'] + ning_values['impact'] + ling_values['impact']
                    if tot_cst <= budget:
                        combinations.append({ingredients[0] +'_'+'source': ting_source,
                                             ingredients[1] +'_'+ 'source': ning_source,
                                             ingredients[2] +'_'+ 'source': ling_source,
                                             'total cost': tot_cst,
                                             'total impact': tot_impt})
        combinations_df = pd.DataFrame(combinations)
        combinations_sorted = combinations_df.sort_values(by=['total cost'])
        for i, row_i in combinations_sorted.iterrows():
            comb_dominated = False
            for j, row_j in combinations_sorted.iterrows():
                if (row_j['total cost'] <= row_i['total cost'] and row_j['total impact'] <= row_i['total impact'] and
                    (row_j['total cost'] < row_i['total cost'] or row_j['total impact'] < row_i['total impact'])):
                    comb_dominated = True
                    break
            if not comb_dominated:
                pareto_front.append(row_i)
        out_df = pd.DataFrame(pareto_front)
        out_df.reset_index(drop=True, inplace=True)
        return out_df

    # Case 4: Four ingredients
    elif len(ingredients) == 4:
        combinations = []
        for ting_source, ting_values in data[home_country][ingredients[0]].items():                   
            for ning_source, ning_values in data[home_country][ingredients[1]].items():
                for ling_source, ling_values in data[home_country][ingredients[2]].items():
                    for fing_source, fing_values in data[home_country][ingredients[3]].items():
                        tot_cst = ting_values['cost'] + ning_values['cost'] + ling_values['cost'] + fing_values['cost']      
                        tot_impt = ting_values['impact'] + ning_values['impact'] + ling_values['impact'] + fing_values['impact']
                        if tot_cst <= budget:
                            combinations.append({ingredients[0] +'_'+'source': ting_source,
                                                 ingredients[1] +'_'+ 'source': ning_source,
                                                 ingredients[2] +'_'+ 'source': ling_source,
                                                 ingredients[3] +'_'+ 'source': fing_source,
                                                 'total cost': tot_cst,
                                                 'total impact': tot_impt})
        combinations_df = pd.DataFrame(combinations)
        combinations_sorted = combinations_df.sort_values(by=['total cost'])
        for i, row_i in combinations_sorted.iterrows():
            comb_dominated = False
            for j, row_j in combinations_sorted.iterrows():
                if (row_j['total cost'] <= row_i['total cost'] and row_j['total impact'] <= row_i['total impact'] and
                    (row_j['total cost'] < row_i['total cost'] or row_j['total impact'] < row_i['total impact'])):
                    comb_dominated = True
                    break
            if not comb_dominated:
                pareto_front.append(row_i)
        out_df = pd.DataFrame(pareto_front)
        out_df.reset_index(drop=True, inplace=True)
        return out_df

## Test Cases

### Case 1: Single ingredient (Tortilla).
The single ingredient case is only able to test Tortilla since the burrito builder application defaults to Tortilla as the base ingredient but any source country can be used from the list.

In [3]:
build_burrito_test(burr_data, home_country='Luxembourg', ingredients=['Tortilla'], budget=20)

,cost,impact
Algeria,0.770920,3.108346
Ireland,0.946317,3.082788
Belgium,1.009932,3.021919
Luxembourg,1.060625,2.973404


### Case 2: Two ingredients
This test is completed by selecting another ingredient to accompany the base ingredient, Tortilla. Any country from the list can be selected as the home_country and a budget amount can be entered if the default amount of $10 is not desired.

In [4]:
build_burrito_test(burr_data, home_country='Colombia', ingredients=['Tortilla', 'Chicken'], budget=20)

,Tortilla_source,Chicken_source,total cost,total impact
0,Canada,USA,1.739152,12.695696
1,Peru,USA,1.748235,12.406051
2,Costa Rica,USA,1.962674,12.306034
3,Israel,USA,2.033245,12.237523


### Case 3: Three ingredients
This test is completed by selecting two ingredients to accompany the base ingredient, Tortilla. Any country from the list can be selected as the home_country and a budget amount can be entered if the default amount of $10 is not desired.

In [5]:
build_burrito_test(burr_data, home_country='Madagascar', ingredients=['Tortilla', 'Chicken', 'Cheese'], budget=20)

,Tortilla_source,Chicken_source,Cheese_source,total cost,total impact
0,South Africa,France,United Kingdom,3.279285,16.314815
1,Comoros,France,United Kingdom,3.290721,15.689477
2,Comoros,France,Belgium,3.373879,15.687472
3,Comoros,France,South Africa,3.382677,15.636247
